In [5]:
import os
import requests
import json
import pandas as pd
from datetime import datetime, timezone

In [6]:
# Configuration
API_TOKEN = os.environ.get("WAQI_API_KEY")  # API token for WAQI saved as an environment variable
BASE_URL = "https://api.waqi.info/feed/"

In [7]:
def get_realtime_aqi(city_name):
    """Fetches real-time AQI data for a specific city using WAQI API."""
    url = f"{BASE_URL}{city_name}/?token={API_TOKEN}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        waqi_data = response.json()

        if waqi_data['status'] == 'ok':
            data = waqi_data['data']
            aqi = data['aqi']
            pollutants = data['iaqi'] # Individual pollutant data

            # Create a dictionary to store the data
            aqi_dict = {'timestamp': datetime.now(timezone.utc), 'aqi': aqi}
            for pollutant, values in pollutants.items():
                if 'v' in values:
                    aqi_dict[pollutant] = values['v']

            aqi_df = pd.DataFrame([aqi_dict])
            aqi_df['timestamp'] = pd.to_datetime(aqi_df['timestamp'])
            aqi_df.set_index('timestamp', inplace=True)
            

            print(f"WAQI Data for {city_name} was ingested successfully!")
            return aqi_df

        else:
            print(f"Error fetching WAQI data: {waqi_data['message']}")

    except requests.exceptions.RequestException as e:
        print(f"Error connecting to WAQI API: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding WAQI JSON response: {e}")
    except KeyError as e:
        print(f"Error accessing key in WAQI data: {e}")

In [8]:
# Example Usage
city = "Paris" # Or use specific station ID like "@1437" for a specific city station
real_time_data = get_realtime_aqi(city)

WAQI Data for Paris was ingested successfully!


In [9]:
real_time_data

,aqi,co,h,no2,o3,p,pm10,pm25,so2,t,w
timestamp,,,,,,,,,,,
2025-04-09 12:50:55.184099+00:00,40,0.1,40,24.7,18.9,1024.7,40,88,0.6,16.3,4.3


In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\pc\\Desktop\\GitHub repos\\End-to-end-ML-project-with-MLflows'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from mlProject.constants.__init__ import *
from mlProject.utils.common import read_yaml, create_dirs

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-09 16:19:19,564: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-09 16:19:19,570: INFO: common: yaml file: params.yaml loaded successfully]


[2025-04-09 16:19:19,576: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-09 16:19:19,580: INFO: common: created directory at: artifacts]
[2025-04-09 16:19:19,583: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-09 16:19:20,955: INFO: 2280763589: artifacts/data_ingestion/Paris_daily_air_quality.zip download! with following info: 
Connection: close
Content-Length: 30811
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "45708a86bfad6052728b7aa0dc4640aa1e872cae0c1075a7320e4fe9f2463346"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EEAB:24FA2C:43F6BC:489C41:67F681E7
Accept-Ranges: bytes
Date: Wed, 09 Apr 2025 14:19:20 GMT
Via: 1.1 varnish
X-Served-By: cache-par-lfpg1960081-PAR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1744208360.011495,VS0,VE197
Vary: Aut